In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re

In [2]:
train_data = pd.read_csv("train_data.csv",names=["sentiments", "reviews"])

In [4]:
def basic_cleaning(data_frame):
    data_frame['reviews'] = data_frame.reviews.apply(lambda review: review.lower())
    data_frame['reviews'] = data_frame.reviews.apply(lambda review: review.translate(str.maketrans('', '', string.punctuation)))
    data_frame['reviews'] = data_frame.reviews.apply(lambda review: review.translate(str.maketrans('', '', string.digits)))
    data_frame['reviews'] = data_frame.reviews.apply(lambda review: re.sub("r[^a-z]",'',review))

    
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

def tokenize_data(data_frame):
    data_frame['words'] = data_frame.reviews.apply(lambda review: nltk.word_tokenize(review))

from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

def remove_stopwords(list):
    stop_words_removed = []
    for i in list:
        if i not in stopwords:
            stop_words_removed.append(i)
    return stop_words_removed

In [38]:
basic_cleaning(train_data)
tokenize_data(train_data)
train_data['stop_words_cleaned'] = train_data.words.apply(lambda word_list: remove_stopwords(word_list))

In [39]:
# nltk.download('all')
def tag_pos(list_of_words):
    return nltk.pos_tag(list_of_words)

#extraction of lemma words after taggin with pos
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()
def find_lemma_word(word):
    lemma_words=[]
    words_with_pos = tag_pos(word)
    for word in words_with_pos:
        if word[1].startswith('NN'):
            lemma_words.append(lemmatizer.lemmatize(word[0],pos='n'))
        elif word[1].startswith('VB'):
            lemma_words.append(lemmatizer.lemmatize(word[0],pos='v'))
        elif word[1].startswith('JJ'):
            lemma_words.append(lemmatizer.lemmatize(word[0],pos='a'))
        elif word[1].startswith('RB'):
            lemma_words.append(lemmatizer.lemmatize(word[0],pos='r'))
        else:
            lemma_words.append(word[0])
            
    return lemma_words

In [40]:
train_data['lemma_word'] = train_data.stop_words_cleaned.apply(lambda word_list: find_lemma_word(word_list))
train_data['cleaned_review'] = train_data.lemma_word.apply(lambda review_list: " ".join(review_list))

In [41]:
train_data

,sentiments,reviews,words,stop_words_cleaned,lemma_word,cleaned_review
0,-1,eat at fioris they said youll like it they sa...,"[eat, at, fioris, they, said, youll, like, it,...","[eat, fioris, said, youll, like, saidnnis, con...","[eat, fioris, say, youll, like, saidnnis, conv...",eat fioris say youll like saidnnis convenientl...
1,-1,i just dont understand the appeal ive tried t...,"[i, just, dont, understand, the, appeal, ive, ...","[dont, understand, appeal, ive, tried, place, ...","[dont, understand, appeal, ive, tried, place, ...",dont understand appeal ive tried place twice t...
2,1,this is my go to place foa really good beef en...,"[this, is, my, go, to, place, foa, really, goo...","[go, place, foa, really, good, beef, enchilada...","[go, place, foa, really, good, beef, enchilada...",go place foa really good beef enchilada red sa...
3,-1,not impressed when i ordered the oyako bowl th...,"[not, impressed, when, i, ordered, the, oyako,...","[impressed, ordered, oyako, bowl, conversation...","[impressed, order, oyako, bowl, conversation, ...",impressed order oyako bowl conversation go som...
4,-1,this is the first time evei wrote a bad review...,"[this, is, the, first, time, evei, wrote, a, b...","[first, time, evei, wrote, bad, review, frustr...","[first, time, evei, write, bad, review, frustr...",first time evei write bad review frustrate her...
...,...,...,...,...,...,...
17995,-1,i was referred to go to this place by a buddy ...,"[i, was, referred, to, go, to, this, place, by...","[referred, go, place, buddy, aftea, conversati...","[refer, go, place, buddy, aftea, conversation,...",refer go place buddy aftea conversation get sh...
17996,1,the food here was really good we started off ...,"[the, food, here, was, really, good, we, start...","[food, really, good, started, garlic, bread, c...","[food, really, good, start, garlic, bread, cov...",food really good start garlic bread cover toma...
17997,1,i eat at this place maybe a week i am die har...,"[i, eat, at, this, place, maybe, a, week, i, a...","[eat, place, maybe, week, die, hard, wing, fan...","[eat, place, maybe, week, die, hard, wing, fan...",eat place maybe week die hard wing fan best ev...
17998,1,phoenix airport is getting betteday by day i ...,"[phoenix, airport, is, getting, betteday, by, ...","[phoenix, airport, getting, betteday, day, pri...","[phoenix, airport, get, betteday, day, primari...",phoenix airport get betteday day primarily use...


In [42]:
from sklearn.model_selection import train_test_split
train_review, test_review, train_sentiment, test_sentiment =train_test_split(train_data.cleaned_review,train_data.sentiments,shuffle=True,random_state=0,stratify=train_data.sentiments,train_size=.96)

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
number_of_train_documents = len(train_review)
min_doc_percentage = max(5/number_of_train_documents, (len(train_review)*0.0005)/number_of_train_documents)
tf_vectorizer = TfidfVectorizer(min_df=min_doc_percentage,use_idf=False)
feature_matrix = tf_vectorizer.fit_transform(train_review)
feature_array = feature_matrix.toarray()

In [83]:
tf_vectorizer.vocabulary_['able']

4

### Select K-best features

In [84]:
from sklearn.feature_selection import SelectKBest, chi2

vocab_list = tf_vectorizer.get_feature_names_out()

select_k_best = SelectKBest(score_func=chi2, k= int(len(vocab_list)*.20))
train_sentiment_np_array = np.array(train_sentiment)
select_k_best.fit(feature_array, train_sentiment_np_array)
mask = select_k_best.get_support()
k_best_feature = vocab_list[mask]
k_best_feature

array(['able', 'absurd', 'abysmal', ..., 'yummy', 'zero', 'zoo'],
      dtype=object)

### Supervised Chi-Square weight for k-best term by utilising target value

In [224]:
from sklearn.feature_extraction.text import CountVectorizer
selected_tf_vectorizer = CountVectorizer(vocabulary=k_best_feature)
selected_feat_array = selected_tf_vectorizer.fit_transform(train_review).toarray()
selected_test_array = selected_tf_vectorizer.transform(test_review).toarray()
observed_value_table = pd.DataFrame(k_best_feature, columns=['features'])

In [225]:
tf_data_frame = pd.DataFrame(selected_feat_array,columns = selected_tf_vectorizer.get_feature_names_out())
tf_data_frame['sentiments'] = np.array(train_sentiment)

In [226]:
observed_value_table['positive_sentiment'] = observed_value_table.features.apply(lambda feature: tf_data_frame.loc[tf_data_frame['sentiments']==1,feature].sum())

In [227]:
observed_value_table['negative_sentiment'] = observed_value_table.features.apply(lambda feature: tf_data_frame.loc[tf_data_frame['sentiments']==-1,feature].sum())

In [228]:
observed_value_table['total_row_count'] = observed_value_table['positive_sentiment'] + observed_value_table['negative_sentiment']

In [229]:
tf_data_frame

,able,absurd,abysmal,accept,acceptable,accessory,accommodate,accomodating,account,accuse,...,youpet,yourestaurant,youtime,yuck,yucky,yum,yummy,zero,zoo,sentiments
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,-1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1


In [230]:
observed_value_table

,features,positive_sentiment,negative_sentiment,total_row_count
0,able,324,320,644
1,absurd,1,19,20
2,abysmal,0,17,17
3,accept,27,88,115
4,acceptable,11,57,68
...,...,...,...,...
1512,yucky,1,8,9
1513,yum,113,15,128
1514,yummy,257,47,304
1515,zero,16,160,176


In [231]:
chi_sqare_expected = pd.DataFrame()

#### Calculating expected value table for chi-sqare function

In [232]:
def chi_sqare_value(row_totals, positive_column_total, negative_column_total, total_sum):
    positive_chi_value = []
    negative_chi_value = []
    for i in range(0, len(row_totals)):
        positive_chi_value.append((row_totals[i]*positive_column_total)/total_sum)
        negative_chi_value.append((row_totals[i]*negative_column_total)/total_sum)
    return positive_chi_value, negative_chi_value

In [233]:
row_totals = np.array(observed_value_table['total_row_count'])
positive_column_total = observed_value_table['positive_sentiment'].sum()
negative_column_total = observed_value_table['negative_sentiment'].sum()
total_sum = observed_value_table['total_row_count'].sum()
expected_positive, expected_negative = chi_sqare_value(row_totals, positive_column_total, negative_column_total, total_sum)

In [234]:
chi_sqare_expected['+1'] = np.array(expected_positive)
chi_sqare_expected['-1'] = np.array(expected_negative)

In [235]:
chi_sqare_expected

,+1,-1
0,267.166988,376.833012
1,8.297111,11.702889
2,7.052545,9.947455
3,47.708391,67.291609
4,28.210179,39.789821
...,...,...
1512,3.733700,5.266300
1513,53.101513,74.898487
1514,126.116094,177.883906
1515,73.014580,102.985420


In [236]:
chi_sqare_table = pd.DataFrame()

In [237]:
chi_sqare_table['positiev_sentiments'] = ((observed_value_table['positive_sentiment'] - chi_sqare_expected['+1'])**2)/chi_sqare_expected['+1']
chi_sqare_table['negatiev_sentiments'] = ((observed_value_table['negative_sentiment'] - chi_sqare_expected['-1'])**2)/chi_sqare_expected['-1']


In [238]:
chi_sqare_table

,positiev_sentiments,negatiev_sentiments
0,12.089784,8.571413
1,6.417635,4.549974
2,7.052545,5.000112
3,8.988722,6.372822
4,10.499411,7.443870
...,...,...
1512,2.001531,1.419045
1513,67.565471,47.902553
1514,135.831966,96.302118
1515,44.520729,31.564297


### Create weighted features

In [239]:
def generate_chi_sqare_weighted_features(train_feat_array, train_sentiments, p_chi_weight, n_chi_weight):
    new_weighted_feature= []
    sentiments = np.array(train_sentiments)
    for i in range(0,len(sentiments)):        
        if sentiments[i] == 1:
            new_weighted_feature.append(np.multiply(train_feat_array[i],p_chi_weight))
        else:
            new_weighted_feature.append(np.multiply(train_feat_array[i],n_chi_weight))
    return new_weighted_feature        

In [240]:
p_chi_weight = np.array(chi_sqare_table['positiev_sentiments'])
n_chi_weight = np.array (chi_sqare_table['negatiev_sentiments'])
weighted_selected_feat = np.array(generate_chi_sqare_weighted_features(selected_feat_array, train_sentiment,p_chi_weight,n_chi_weight))

In [242]:
test_sentiments_array = np.array(test_sentiment)
train_sentiments_array = np.array(train_sentiment)
weighted_test_feat = np.array(generate_chi_sqare_weighted_features( selected_test_array, test_sentiment,p_chi_weight,n_chi_weight))


In [243]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
track_precision=[]
for i in range(3,21):
    print(i)
    classifer = KNeighborsClassifier(n_neighbors=i,n_jobs=3,weights='distance')
    classifer.fit(weighted_selected_feat, train_sentiments_array)
    sentiments_predict = classifer.predict(weighted_test_feat)
    score = metrics.accuracy_score(test_sentiments_array,sentiments_predict)
    print(score)
    track_precision.append(score)

3
0.8972222222222223
4
0.9027777777777778
5
0.8972222222222223
6
0.9013888888888889
7
0.9041666666666667
8
0.9055555555555556
9
0.9041666666666667
10
0.9083333333333333
11
0.8972222222222223
12
0.8972222222222223
13
0.9027777777777778
14
0.9
15
0.9013888888888889
16
0.9041666666666667
17
0.8972222222222223
18
0.8972222222222223
19
0.8972222222222223
20
0.8958333333333334
